# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

In [4]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [5]:
# frequency table for prestige and whether or not someone was admitted
df.groupby(['prestige','admit']).count()

gre  gpa
prestige admit          
1.0      0       28   28
         1       33   33
2.0      0       95   95
         1       53   53
3.0      0       93   93
         1       28   28
4.0      0       55   55
         1       12   12

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [63]:
prestige_dummy = pd.get_dummies(df['prestige'], prefix = 'p')
print prestige_dummy.head()

   p_1.0  p_2.0  p_3.0  p_4.0
0    0.0    0.0    1.0    0.0
1    0.0    0.0    1.0    0.0
2    1.0    0.0    0.0    0.0
3    0.0    0.0    0.0    1.0
4    0.0    0.0    0.0    1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: one, for prestige.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [22]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummy.ix[:, 'p_1':])
handCalc.head()

,admit,gre,gpa,p_1.0,p_2.0,p_3.0,p_4.0
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


In [23]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['admit'], handCalc['p_1.0'], rownames=['admit'])

p_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [27]:
odds_admission = 33.0/28
odds_admission

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [28]:
odds_admission2 = 93.0 / 243
odds_admission2

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [30]:
odds_ratio = odds_admission / odds_admission2
odds_ratio

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of getting into a 3.07 times higher if a student attends a #1 ranked college

#### 3.5 Print the cross tab for prestige_4

In [31]:
pd.crosstab(handCalc['admit'], handCalc['p_4.0'], rownames=['admit'])

p_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [35]:
odds_admission = 12.0 / 55
odds_admission2 = 114.0 / 216
OR = odds_admission / odds_admission2
OR

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: If a student attends a 4th tier ranked school, they are 40% less likely to get admitted to graduate school compared to a student who did not attend a 4th tier ranked school

## Part 4. Analysis

In [38]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummy.ix[:, 'p_2':])
print data.head()

   admit    gre   gpa  p_2.0  p_3.0  p_4.0
0      0  380.0  3.61    0.0    1.0    0.0
1      1  660.0  3.67    0.0    1.0    0.0
2      1  800.0  4.00    0.0    0.0    0.0
3      1  640.0  3.19    0.0    0.0    1.0
4      0  520.0  2.93    0.0    0.0    1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [39]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [42]:
train_cols = data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'p_2.0', u'p_3.0', u'p_4.0', u'intercept'], dtype='object')

#### 4.2 Fit the model

In [43]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [46]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 18 Sep 2016   Pseudo R-squ.:                 0.08166
Time:                        15:10:50   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
p_2.0         -0.6801      0.317     -2.146      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [47]:
print np.exp(result.params)

gre          1.002221
gpa          2.180027
p_2.0        0.506548
p_3.0        0.262192
p_4.0        0.211525
intercept    0.020716
dtype: float64


In [48]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

               2.5%     97.5%        OR
gre        1.000074  1.004372  1.002221
gpa        1.136120  4.183113  2.180027
p_2.0      0.272168  0.942767  0.506548
p_3.0      0.133377  0.515419  0.262192
p_4.0      0.093329  0.479411  0.211525
intercept  0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: A student who attented a 2nd tier ranked school is 50% chance of getting admitted into graduate school in comparison to a student who attended a 1st tier ranked school

#### 4.6 Interpret the OR of GPA

Answer: If a student's GPA increases by one unit, he or she is 1.18x more likely to get admitted into graduate school.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [49]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [50]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

* This part I wasn't sure about so I just looked at the Solution code. 

In [60]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
prestige_dummy = pd.get_dummies(combos['prestige'], prefix='p')
prestige_dummy.columns = ['p_1.0', 'p_2.0', 'p_3.0', 'p_4.0']

# keep only what we need for making predictions
keep_cols = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[keep_cols].join(prestige_dummy.ix[:, 'p_2.0':])

#### 5.2 Make predictions on the enumerated dataset

In [64]:
combos['prediction'] = result.predict(combos[train_cols])

print combos
len(combos)

       gre       gpa  prestige  intercept  p_2.0  p_3.0  p_4.0  prediction
0    220.0  2.260000       1.0        1.0    0.0    0.0    0.0    0.164173
1    220.0  2.260000       2.0        1.0    1.0    0.0    0.0    0.090492
2    220.0  2.260000       3.0        1.0    0.0    1.0    0.0    0.048977
3    220.0  2.260000       4.0        1.0    0.0    0.0    1.0    0.039890
4    220.0  2.453333       1.0        1.0    0.0    0.0    0.0    0.185907
5    220.0  2.453333       2.0        1.0    1.0    0.0    0.0    0.103682
6    220.0  2.453333       3.0        1.0    0.0    1.0    0.0    0.056492
7    220.0  2.453333       4.0        1.0    0.0    0.0    1.0    0.046078
8    220.0  2.646667       1.0        1.0    0.0    0.0    0.0    0.209795
9    220.0  2.646667       2.0        1.0    1.0    0.0    0.0    0.118543
10   220.0  2.646667       3.0        1.0    0.0    1.0    0.0    0.065080
11   220.0  2.646667       4.0        1.0    0.0    0.0    1.0    0.053173
12   220.0  2.840000     

400

#### 5.3 Interpret findings for the last 4 observations

Answer: According to the last four observations, a student who attended a ranked #1 undergraduate school has a 73% chance of getting into graduate school, while a student who attended a ranked #4 school has a 37% chance of being admitted.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.